# Preprocess validation dataset

In [1]:
import sys, os, time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io
import cv2
from keras import backend as K
from keras.utils import to_categorical
from keras.applications.vgg19 import VGG19, preprocess_input

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def humanSize(nbytes):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    i = 0
    while nbytes >= 1024 and i < len(suffixes) - 1:
        nbytes /= 1024
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])


# load data as a list
fns = os.listdir("data/val")
fns.sort()
fns = [
    "data/val/" + fn
    for fn in fns
]


print('file 1:', fns[0])


file 1: data/val/ILSVRC2012_val_00000001.JPEG


In [2]:
def humansize(nbytes):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])



## Match labels (done)

In [8]:
# match meta.mat and synset_words.txt
meta = scipy.io.loadmat("data/meta.mat")
original_idx_to_synset = {}
synset_to_name = {}

for i in range(1000):
    ilsvrc2012_id = int(meta['synsets'][i,0][0][0][0])
    synset = meta["synsets"][i,0][1][0]
    name = meta["synsets"][i,0][2][0]
    original_idx_to_synset[ilsvrc2012_id] = synset
    synset_to_name[synset] = name

# assign each synset word an id from 1 to 100
synset_to_keras_idx = {}
keras_idx_to_name = {}


f = open("data/synset_words.txt","r")
idx = 0
for line in f:
    parts = line.split(" ")
    synset_to_keras_idx[parts[0]] = idx
    keras_idx_to_name[idx] = " ".join(parts[1:])
    idx += 1

def convert_original_idx_to_keras_idx(idx):
    return synset_to_keras_idx[original_idx_to_synset[idx]]


## First 5000

In [3]:
%%time

x_val5 = np.zeros((5000, 224, 224, 3), dtype=np.float32)
print(humansize(x_val5.nbytes))

x_val5_2 = np.zeros((5000, 299, 299, 3), dtype=np.float32)
print(humansize(x_val5_2.nbytes))


2.8 GB
5 GB
CPU times: user 238 µs, sys: 338 µs, total: 576 µs
Wall time: 364 µs


In [4]:
%%time


for i in range(5000):
    if i % 1000 == 0:    # for checking progress
        print("%d/%d" % (i, 5000))
        
        
    # load as BGR
    img = cv2.imread(fns[i])
    
    # resize
    height, width = img.shape[:2]
    new_height =  height * 256 // min(img.shape[:2])
    new_width = width * 256 // min(img.shape[:2])
    img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_CUBIC)
    
    # crop
    height, width = img.shape[:2]
    startx = width//2 - (224//2)
    starty = height//2 - (224//2)
    img = img[starty:starty+224,startx:startx+224]
    assert img.shape[0] == 224 and img.shape[1] == 224
    
    # save as rgb
    x_val5[i,:,:,:] = img[:,:,::-1]
    


0/5000
1000/5000
2000/5000
3000/5000
4000/5000
CPU times: user 1min 12s, sys: 17.6 s, total: 1min 29s
Wall time: 18.7 s


In [5]:
%%time

np.save("data/x_val5.npy", x_val5)

CPU times: user 76.3 ms, sys: 1.18 s, total: 1.26 s
Wall time: 2.23 s


In [17]:
%%time

for i in range(5000):
    if i % 1000 == 0:    # for checking progress
        print("%d/%d" % (i, 5000))
        
    # load as BGR
    img = cv2.imread(fns[i])
    
    # resize
    height, width = img.shape[:2]
    new_height =  height * 299 // min(img.shape[:2])
    new_width = width * 299 // min(img.shape[:2])
    img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_CUBIC)
    
    # crop
    height, width = img.shape[:2]
    startx = width//2 - (299//2)
    starty = height//2 - (299//2)
    img = img[starty:starty+299,startx:startx+299]
    assert img.shape[0] == 299 and img.shape[1] == 299, (img.shape, height, width)
    
    # save as rgb
    x_val5_2[i,:,:,:] = img[:,:,::-1]
    


0/1000
1000/1000
2000/1000
3000/1000
4000/1000
CPU times: user 1min 16s, sys: 20.8 s, total: 1min 36s
Wall time: 1min 14s


In [18]:
%%time

np.save("data/x_val5_2.npy", x_val5_2)

CPU times: user 0 ns, sys: 2.1 s, total: 2.1 s
Wall time: 5.34 s


In [10]:
# create y_val from ground truth text file

f = open("data/ILSVRC2012_validation_ground_truth.txt","r")
y_val5 = f.read().strip().split("\n")
y_val5 = list(map(int, y_val5))
y_val5 = np.array([convert_original_idx_to_keras_idx(idx) for idx in y_val5[:5000]])
f.close()


In [11]:
np.save("data/y_val5.npy", y_val5)

## First 1000

In [4]:
%%time

x_val1 = np.zeros((1000, 224, 224, 3), dtype=np.float32)
print(humansize(x_val1.nbytes))

x_val1_2 = np.zeros((1000, 299, 299, 3), dtype=np.float32)
print(humansize(x_val1_2.nbytes))


574.22 MB
1023.11 MB
CPU times: user 129 µs, sys: 188 µs, total: 317 µs
Wall time: 187 µs


In [6]:
%%time

# done 

for i in range(1000):
    if i % 200 == 0:    # for checking progress
        print("%d/%d" % (i, 1000))
        
        
    # load as BGR
    img = cv2.imread(fns[i])
    
    # resize
    height, width = img.shape[:2]
    new_height =  height * 256 // min(img.shape[:2])
    new_width = width * 256 // min(img.shape[:2])
    img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_CUBIC)
    
    # crop
    height, width = img.shape[:2]
    startx = width//2 - (224//2)
    starty = height//2 - (224//2)
    img = img[starty:starty+224,startx:startx+224]
    assert img.shape[0] == 224 and img.shape[1] == 224
    
    # save as rgb
    x_val1[i,:,:,:] = img[:,:,::-1]
    


0/1000
200/1000
400/1000
600/1000
800/1000
CPU times: user 14.6 s, sys: 3.5 s, total: 18.1 s
Wall time: 3.58 s


In [8]:
%%time

np.save("data/x_val1.npy", x_val1)

CPU times: user 0 ns, sys: 245 ms, total: 245 ms
Wall time: 544 ms


In [10]:
%%time

for i in range(1000):
    if i % 200 == 0:    # for checking progress
        print("%d/%d" % (i, 1000))
        
    # load as BGR
    img = cv2.imread(fns[i])
    
    # resize
    height, width = img.shape[:2]
    new_height =  height * 299 // min(img.shape[:2])
    new_width = width * 299 // min(img.shape[:2])
    img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_CUBIC)
    
    # crop
    height, width = img.shape[:2]
    startx = width//2 - (299//2)
    starty = height//2 - (299//2)
    img = img[starty:starty+299,startx:startx+299]
    assert img.shape[0] == 299 and img.shape[1] == 299, (img.shape, height, width)
    
    # save as rgb
    x_val1_2[i,:,:,:] = img[:,:,::-1]
    


0/1000
200/1000
400/1000
600/1000
800/1000
CPU times: user 15.1 s, sys: 3.42 s, total: 18.5 s
Wall time: 3.69 s


In [12]:
%%time

np.save("data/x_val1_2.npy", x_val1_2)

CPU times: user 0 ns, sys: 420 ms, total: 420 ms
Wall time: 793 ms


In [26]:
# create y_val from ground truth text file

f = open("data/ILSVRC2012_validation_ground_truth.txt","r")
y_val = f.read().strip().split("\n")
y_val = list(map(int, y_val))
y_val = np.array([convert_original_idx_to_keras_idx(idx) for idx in y_val[:1000]])
f.close()


In [27]:
np.save("data/y_val1.npy", y_val)